<a href="https://colab.research.google.com/github/badmsconfig/DZ_11/blob/main/KIAv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not in

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


HISTORY = []



In [ ]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-16k', 0.003, 0.004]
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_3_5_TURBO_INSTRUCT = ['gpt-3.5-turbo-instruct', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
MODEL_GPT_4t = ['gpt-4-1106-preview', 0.03, 0.06]  # 8,192 tokens

SELECT_MODEL_GPT = MODEL_GPT_4t

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def insert_newlines(textstr: str, max_len: int = 170) -> str:
    words = textstr.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
import re

def load_file_knowledge(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "router"),
        ("##", "theme"),
        ("###", "Header3"),
        ("####", "Header4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    # Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())

    return vectordateBase



In [ ]:
knowledge_base = load_file_knowledge('database.md')

In [ ]:

def _summarize_topic( dialog):
        messages = [
            {"role": "system",
             "content": "Ты - ассистент консультанта, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги консультанта и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
            {"role": "user",
             "content": "Суммаризируй следующий диалог консультанта и клиента: " + " ".join(dialog)}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=0.1,  # Используем более низкую температуру для более определенной суммаризации
            max_tokens=1000  # Ограничиваем количество токенов для суммаризации
        )

        return completion.choices[0].message.content



def answer_index(topic, temp=0.1, top_similar_documents=5):
        summarize_text = ''
        HISTORY = '' # ОТКЛЮЧАЕТ САММОРИЗАЦИЮ! ЕСТЬ НУЖНО ЕЕ ВКЛЮЧИТЬ ТОГДА ЗАКОМЕНТИРУЙ
        if len(HISTORY) > 0:
            summarize_text = "Вот краткий обзор предыдущего диалога: " + _summarize_topic(
                [q + ' ' + (a if a is not None else '') for q, a in HISTORY])
            print(f'САММАРИ \n=== {summarize_text} \n')

        # Добавляем явное разделение между историей диалога и текущим вопросом
        input_text = summarize_text + "\n\nТекущий вопрос: " + topic

        docs = knowledge_base.similarity_search_with_score(topic, k=top_similar_documents)
        responses = []
        for i, (doc, score) in enumerate(docs):
            if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
                content = doc.page_content
                response = f'\n=====================Отрывок документа №{i + 1}=====================\n{content}\n'
                print(f'\n=====================Отрывок документа №{i + 1}=====================\n')
                print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')
                print(f'\n{content}\n')
                responses.append(response)


        messages = [
            {"role": "system", "content": prompt},
            {"role": "user",
             "content": f"Документ с информацией для ответа пользователю: {responses}\n\nВопрос клиента: \n{input_text}"}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=temp
        )

        answer = completion.choices[0].message.content

        # Добавляем вопрос пользователя и ответ системы в историю
        if len(HISTORY) > 0:
            HISTORY.append((topic, answer if answer is not None else ''))

        return insert_newlines(answer)  # возвращает ответ


In [ ]:
def run_dialog(system_doc_url, knowledge_base_url):

    while True:
        user_question = input('\nКлиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer = answer_index(user_question)
        print('\nМенеджер: ', answer+'\n\n')

    return

In [ ]:
# Промпт моделей машин
prompt = load_document_text('https://docs.google.com/document/d/1i8HA7cX4Ut-tb9rf8wOgERU7lLe66xJYscizGtSSJl0')

In [ ]:
temperature=0.3
verbose=0
#HISTORY = [] # Убери решетку если нужно обновлять историю диалогов после запуска, если не нужно обновлять то оставь так

run_dialog(prompt, knowledge_base)


Клиент: "Здравствуйте подскажите пожалуйста мне с банка пришло смс о первоначальном одобрение Я оставлял заявку на покупку машины"

=====================Отрывок документа №1=====================

=== score = 0.28271329402923584  Metadata документа ------------ {'router': 'sales', 'theme': 'sales_purchases187'}

Информацию о покупке автомобиля в рассрочку или кредит под такси без первоначального взноса можно получить у официальных дилерских центров и банков-партнеров.


=====================Отрывок документа №2=====================

=== score = 0.30932557582855225  Metadata документа ------------ {'router': 'apps', 'theme': 'Проблема с активацией автомобиля в приложении Mobikey'}

Если автомобиль не отображается в приложении mobikey, необходимо направить обращение на электронную почту mobikey@hyundai.ru, указав следующую информацию: имя и фамилию владельца учетной записи, телефон, указанный дилеру для регистрации, VIN номер автомобиля, дилерский центр, который производил активацию, опи